In [1]:
import os
import torch 
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0,1)
    x = x.view(x.size(0), 1, 28,28)
    return x

In [13]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [14]:
dataloader.batch_size

128

In [15]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128,64),
            nn.ReLU(True),
            nn.Linear(64,12),
            nn.ReLU(True),
            nn.Linear(12,3),
        )
        self.decoder = nn.Sequential(
            nn.Linear(3,12),
            nn.ReLU(True),
            nn.Linear(12,64),
            nn.ReLU(True),
            nn.Linear(64,128),
            nn.ReLU(True),
            nn.Linear(128,28 * 28),
        )
        
    def forward(self,x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = autoencoder().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),lr=learning_rate,weight_decay=1e-5,
)

for epoch in range(num_epochs):
    for data in dataloader:
        img,_ = data
        img = img.view(img.size(0),-1)
        img = Variable(img).to(device)
        #=====================FORWARD PASS================================
        output = model(img)
        loss = loss_fn(output,img)
        
        #+++++++++++++++++++++BACKWARD PASS++++++++++++++++++++++++++++++++
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #*********************LOGGING**************************************
    print('epoch[{}/{}], loss: {:.4f}'.format(epoch + 1,num_epochs,loss.item()))
    if epoch % 10 == 0:
        if torch.cuda.is_available():
            pic = to_img(output.cpu().data)
        else:
            pic = to_img(output.data)
        save_image(pic,'./mlp_img/image_{}.png'.format(epoch))



epoch[1/100], loss: 0.1988
epoch[2/100], loss: 0.1805
epoch[3/100], loss: 0.1841
epoch[4/100], loss: 0.1668
epoch[5/100], loss: 0.1617
epoch[6/100], loss: 0.1495
epoch[7/100], loss: 0.1636
epoch[8/100], loss: 0.1548
epoch[9/100], loss: 0.1605
epoch[10/100], loss: 0.1404
epoch[11/100], loss: 0.1488
epoch[12/100], loss: 0.1532
epoch[13/100], loss: 0.1539
epoch[14/100], loss: 0.1514
epoch[15/100], loss: 0.1448
epoch[16/100], loss: 0.1544
epoch[17/100], loss: 0.1450
epoch[18/100], loss: 0.1371
epoch[19/100], loss: 0.1491
epoch[20/100], loss: 0.1429
epoch[21/100], loss: 0.1336
epoch[22/100], loss: 0.1414
epoch[23/100], loss: 0.1393
epoch[24/100], loss: 0.1561
epoch[25/100], loss: 0.1470
epoch[26/100], loss: 0.1415
epoch[27/100], loss: 0.1454
epoch[28/100], loss: 0.1495
epoch[29/100], loss: 0.1327
epoch[30/100], loss: 0.1390
epoch[31/100], loss: 0.1323
epoch[32/100], loss: 0.1475
epoch[33/100], loss: 0.1389
epoch[34/100], loss: 0.1380
epoch[35/100], loss: 0.1494
epoch[36/100], loss: 0.1369
e

In [ ]:
from pathlib import Path

MODEL_PATH = Path("saved_models")
MODEL_PATH.mkdir(parents=True,exist_ok=True)

#Create model save path
MODEL_NAME = "simple_autoencoder.pth"
MODEL_SAVE_PATH= MODEL_PATH / MODEL_NAME

#Save the state dict
torch.save(obj=model.state_dict(),
           f=MODEL_SAVE_PATH)